### Use `RGBD` camera

In [2]:
import mujoco,cv2,pyvista
import numpy as np
import matplotlib.pyplot as plt
# from pyntcloud import PyntCloud
import sys
sys.path.append('../../')
from utils.mujoco_parser import MuJoCoParserClass
from utils.util import sample_xyzs,rpy2r,r2rpy,compute_view_params,get_interp_const_vel_traj
np.set_printoptions(precision=2,suppress=True,linewidth=100)
plt.rc('xtick',labelsize=6); plt.rc('ytick',labelsize=6)
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
print ("MuJoCo version:[%s]"%(mujoco.__version__))

MuJoCo version:[2.3.4]


### Parse `UR5e` with `RG2` gripper

In [3]:
xml_path = '../../asset/ur5e/scene_ur5e_rg2_d435i_obj_realworld.xml'
# xml_path = '../../asset/ur5e/scene_ur5e_rg2_d435i_vbm.xml'
env = MuJoCoParserClass(name='UR5e with RG2 gripper',rel_xml_path=xml_path,VERBOSE=True)

obj_cylinder_names = [body_name for body_name in env.body_names
             if body_name is not None and (body_name.startswith("obj_cylinder"))]
obj_box_names = [body_name for body_name in env.body_names
                if body_name is not None and (body_name.startswith("obj_box"))]
obj_glass_names = [body_name for body_name in env.body_names
                if body_name is not None and (body_name.startswith("obj_wine"))]

n_cylinder_obj = 3 # len(obj_cylinder_names)
n_box_obj = 3 # len(obj_box_names)
n_glass_obj = 3 # len(obj_glass_names)

# Place objects
env.place_objects(n_obj=3, obj_names=obj_cylinder_names, x_range=[0.75, 0.85], y_range=[-0.15, 0.15], COLORS=True, VERBOSE=True)
# env.place_objects(n_obj=3, obj_names=obj_box_names, COLORS=True, VERBOSE=True) 
env.place_objects(n_obj=3, obj_names=obj_glass_names, COLORS=False, VERBOSE=True)

# Move tables and robot base
env.model.body('base_table').pos = np.array([0,0,0])
env.model.body('front_object_table').pos = np.array([0.38+0.6,0,0])
env.model.body('right_object_table').pos = np.array([-0.05,-0.80,0])
env.model.body('left_object_table').pos = np.array([-0.05,0.80,0])
env.model.body('base').pos = np.array([0.18,0,0.79])


print ("Ready.")

dt:[0.0020] HZ:[500]
n_dof (=nv):[60]
n_geom:[62]
geom_names:['floor', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 'front_object_table', 'right_object_table', 'left_object_table', 'base_table', 'obj_cylinder_01', 'obj_cylinder_02', 'obj_cylinder_03', 'obj_cylinder_04', 'obj_cylinder_05', 'obj_cylinder_06', 'obj_cylinder_07', 'obj_cylinder_08']
n_body:[31]
body_names:['world', 'base', 'shoulder_link', 'upper_arm_link', 'forearm_link', 'wrist_1_link', 'wrist_2_link', 'wrist_3_link', 'tcp_link', 'camera_mount', 'd435i', 'rg2_gripper_base_link', 'camera_center', 'rg2_gripper_finger1_finger_link', 'rg2_gripper_finger1_inner_knuckle_link', 'rg2_gripper_finger1_finger_tip_link', 'rg2_gripper_finger2_finger_link', 'rg2_gripper_finger2_inner_knuckle

### Manually specify Grasp-pose

In [4]:
env.joint_ranges[env.idxs_forward,0]

array([-3.14, -3.14, -3.14, -3.14, -3.14, -3.14])

### Get grasp pose from Primitive grasp poses

In [5]:
from utils.util import pr2t

def softmax(x):
    # Subtract max(x) to compute the softmax in a numerically stable way
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

def get_grasp_pose_primitive(obj_name, grasp_pose=None, dist_orientation="geodesic"):
    tcp_position = env.get_p_body('tcp_link')
    tcp_orientation = rpy2r(np.radians([0,0,0])) @ rpy2r(np.radians([-180,0,90]))
    grasp_obj_position = env.get_p_body(obj_name)
    grasp_obj_orientation = env.get_R_body(obj_name)
    if grasp_pose == "upright":
        grasp_obj_position[2] += 0.08
        grasp_obj_orientation = grasp_obj_orientation @ rpy2r(np.radians([-90,0,90]))
    elif grasp_pose == "right":
        grasp_obj_position[1] += 0.03
        grasp_obj_position[2] += 0.10
        grasp_obj_orientation = grasp_obj_orientation @ rpy2r(np.radians([-180,0,180]))
    elif grasp_pose == "left":
        grasp_obj_position[1] += 0.03
        grasp_obj_position[2] += 0.07
        grasp_obj_orientation = grasp_obj_orientation @ rpy2r(np.radians([-180,0,0]))
    elif grasp_pose == "forward":
        grasp_obj_position[0] += 0.015
        grasp_obj_position[2] += 0.05
        grasp_obj_orientation = grasp_obj_orientation @ rpy2r(np.radians([-180,0,90]))
    elif grasp_pose == "side":
        rand_position = np.random.uniform(-0.1, 0.1)
        rand_orientation = (rand_position + 0.10) * 180 / 0.2
        grasp_obj_position[1] -= rand_position
        grasp_obj_position[2] += 0.07
        grasp_obj_orientation = grasp_obj_orientation @ rpy2r(np.radians([-180,0,rand_orientation]))
    else:   # Randomly sample grasp pose based on distance [Euclidean + Orientation]
        grasp_pose_primitive = ["upright", "right", "left", "side", "forward"]
        grasp_obj_positions = []
        grasp_obj_orientations = []
        grasp_orientation_dists = []
        for grasp_pose_prim in grasp_pose_primitive:
            grasp_obj_pose = get_grasp_pose_primitive(obj_name, grasp_pose_prim)
            grasp_obj_positions.append(grasp_obj_pose[:3, 3])
            grasp_obj_orientations.append(grasp_obj_pose[:3, :3])
        grasp_dist = grasp_obj_positions - tcp_position
        # Calculate distances between orientations
        for grasp_obj_orientation_ in grasp_obj_orientations:
            if dist_orientation == "geodesic":
                trace_product = np.trace(np.dot(tcp_orientation.T, grasp_obj_orientation_))
                grasp_orientation_dist = np.arccos((trace_product - 1) / 2)
            elif dist_orientation == "frobenius":
                grasp_orientation_dist = np.linalg.norm(tcp_orientation - grasp_obj_orientation_, 'fro')
            grasp_orientation_dists.append(grasp_orientation_dist)
        grasp_orientation_dists = np.array(grasp_orientation_dists)
        grasp_dist = np.linalg.norm(grasp_dist, axis=1)
        grasp_weight = 1 / (grasp_dist + grasp_orientation_dists)
        grasp_pose = np.random.choice(grasp_pose_primitive, p=grasp_weight / np.sum(grasp_weight))
        print(f"grasp_pose: {grasp_pose}, grasp_weight: {grasp_weight}")
        grasp_obj_pose = get_grasp_pose_primitive(obj_name, grasp_pose)
        return grasp_obj_pose

    grasp_obj_pose = pr2t(grasp_obj_position, grasp_obj_orientation)

    return grasp_obj_pose

In [6]:
env.init_viewer(viewer_title='UR5e with RG2 gripper',viewer_width=1200,viewer_height=800,
                viewer_hide_menus=True)
env.update_viewer(azimuth=0.0,distance=3.5,elevation=-60,lookat=[0.4,0.05,0.36],
                  VIS_TRANSPARENT=False,VIS_CONTACTPOINT=True,
                  contactwidth=0.05,contactheight=0.05,contactrgba=np.array([1,0,0,1]),
                  VIS_JOINT=True,jointlength=0.5,jointwidth=0.1,jointrgba=[0.2,0.6,0.8,0.6])
env.reset()

grasp_obj_pose = get_grasp_pose_primitive(obj_name="obj_cylinder_02", grasp_pose="forward", dist_orientation="geodesic")
init_ur_q = np.array([np.deg2rad(-90), np.deg2rad(-130), np.deg2rad(120), np.deg2rad(100), np.deg2rad(45), np.deg2rad(-90)])
q_init_upright = np.array([0,-np.pi/2,0,0,np.pi/2,0])

# minus 0.05 to avoid collision direction of the grasp orientation
pre_grasp_obj_pose = grasp_obj_pose.copy()
unit_vec_y = pre_grasp_obj_pose[:3,1] / np.linalg.norm(pre_grasp_obj_pose[:3,1])
unit_vec_z = pre_grasp_obj_pose[:3,2] / np.linalg.norm(pre_grasp_obj_pose[:3,2])
pre_grasp_obj_pose[:3,3] = pre_grasp_obj_pose[:3,3] - 0.12 * unit_vec_y - 0.10 * unit_vec_z

q_ik_pregrasp = env.solve_ik_repel(
    body_name='tcp_link',p_trgt=pre_grasp_obj_pose[:3,3],R_trgt=pre_grasp_obj_pose[:3,:3],
    IK_P=True,IK_R=True, q_init=q_init_upright,idxs_forward=env.idxs_forward, idxs_jacobian=env.idxs_jacobian,
    RESET=False, DO_RENDER=True, th=1, err_th=1e-3, stepsize=1.0, w_weight=0.3, render_every=1.0, eps = 0.3, repulse = 15, VERBOSE=False)

q_ik = env.solve_ik_repel(
    body_name='tcp_link',p_trgt=grasp_obj_pose[:3,3],R_trgt=grasp_obj_pose[:3,:3],
    IK_P=True,IK_R=True, q_init=q_ik_pregrasp,idxs_forward=env.idxs_forward, idxs_jacobian=env.idxs_jacobian,
    RESET=False, DO_RENDER=True, th=1, err_th=1e-3, stepsize=1.0, w_weight=0.3, render_every=1.0, eps = 0.3, repulse = 15, VERBOSE=False)

print(f"Solved joint q values: {q_ik}")
# Close viewer
env.close_viewer()
print ("Done.")

q_init_upright = np.array([0,-np.pi/2,0,0,np.pi/2,0])
times, q_traj = get_interp_const_vel_traj(np.vstack([q_init_upright,q_ik_pregrasp, q_ik]), vel=np.radians(30), HZ=env.HZ)
print ("Joint trajectory ready. duration:[%.2f]sec"%(times[-1]))

Solved joint q values: [-0.27 -1.07  2.26 -1.19  1.3  -0.  ]
Done.
Joint trajectory ready. duration:[5.86]sec


### Execute it !

In [43]:
# Init viewer
env.init_viewer(viewer_title='UR5e with RG2 gripper',viewer_width=1200,viewer_height=800,
                viewer_hide_menus=True)
env.update_viewer(azimuth=66.08,distance=3.0,elevation=-50,lookat=[0.4,0.18,0.71],
                  VIS_TRANSPARENT=False,VIS_CONTACTPOINT=False,
                  contactwidth=0.05,contactheight=0.05,contactrgba=np.array([1,0,0,1]),
                  VIS_JOINT=True,jointlength=0.25,jointwidth=0.05,jointrgba=[0.2,0.6,0.8,0.6])

pcd_tick = 0

# Reset
env.reset()
q_init_upright = np.array([0,-np.pi/2,0,0,np.pi/2,0])
env.forward(q=q_init_upright, joint_idxs=env.idxs_forward)
tick,max_sec = 0,1000
while env.get_sim_time() <= max_sec:
    # q = q_traj[tick, :]
    # Step
    if np.linalg.norm(env.get_p_body('tcp_link')-grasp_obj_pose[:3,3]) < 0.012:
        q = np.append(q_traj[tick, :],0.0) # close gripper
    else:
        q = np.append(q_traj[tick, :],1.0) # open gripper
    env.step(ctrl=q,ctrl_idxs=[0,1,2,3,4,5,6])

    tick = min(tick + 1,q_traj.shape[0]-1)
    if not env.is_viewer_alive(): break
        
    # Render
    if env.loop_every(HZ=20):
        # Compute some poses
        p_tcp,R_tcp = env.get_pR_body(body_name='tcp_link')
        p_cam,R_cam = env.get_pR_body(body_name='camera_center')
        p_base,R_base = env.get_pR_body(body_name='base')
        # Get PCD from a specific view
        p_ego  = p_cam
        p_trgt = p_cam + R_cam[:,2]
        rgb_img,depth_img,pcd,xyz_img = env.get_egocentric_rgb_depth_pcd(
            p_ego=p_ego,p_trgt=p_trgt,rsz_rate=40,fovy=45,BACKUP_AND_RESTORE_VIEW=True)
        env.plot_T(p=p_base+1.25*R_base[:,2],R=np.eye(3,3),
                    PLOT_AXIS=False,label='[%.4f] err'%(np.linalg.norm(env.get_p_body('tcp_link')-grasp_obj_pose[:3,3])))
        for i in range(len(p_semicircle)):
            env.plot_sphere(p=p_semicircle[i], r=0.005, rgba=[0,1,0,1])
        env.render(render_every=1)
        
# Close viewer
env.close_viewer()
print ("Done. Tick:[%d] Time:[%.2f]sec"%(env.tick,env.get_sim_time()))

Pressed ESC
Quitting.
Done. Tick:[427] Time:[0.85]sec


In [65]:
import numpy as np

obj_name = 'obj_cylinder_01'
p_obj = env.get_p_body(obj_name)
R_obj = env.get_R_body(obj_name)
p_eef = env.get_p_body('tcp_link')
R_eef = env.get_R_body('tcp_link')
R_world = np.eye(3)

unit_xy = p_eef[:2] - p_obj[:2]
unit_xy = np.append(unit_xy, 0)
unit_xy /= np.linalg.norm(unit_xy)
vec_to_eef = p_eef - p_obj
projected_vec = np.dot(vec_to_eef, unit_xy) * unit_xy
# sample region of the semicircle
radius = 0.05
normalized_projected_vec = projected_vec / np.linalg.norm(projected_vec)

num_points = 50
theta = np.linspace(np.pi*0.25, np.pi*0.75, num_points)
p_semicircle = np.zeros((num_points, 3))

# List to store the rotations (pose) at each semicircle_point
R_semicirlce = []
for i in range(num_points):
    point_on_semicircle = p_obj + radius * (normalized_projected_vec) * np.sin(theta[i]) + radius * R_world[:, 1] * np.cos(theta[i])
    point_on_semicircle[2] += 0.07
    p_semicircle[i] = point_on_semicircle

    # Define an arbitrary z-axis for orientation reference
    z_axis = np.array([0, 0, 1])
    rotation_matrix = np.eye(3)
    rotation_matrix[:, 1] = -normalized_projected_vec
    rotation_matrix[:, 0] = np.cross(normalized_projected_vec, z_axis)
    rotation_matrix[:, 0] /= np.linalg.norm(rotation_matrix[:, 1])
    rotation_matrix[:, 2] = np.cross(rotation_matrix[:, 0], rotation_matrix[:, 1])
    R_semicirlce.append(rotation_matrix)

# Convert the list of rotations to a numpy array
R_semicirlce = np.array(R_semicirlce)

import pyvista as pv

# passthrough filter about z axis
def passthrough_filter(pcd, axis, interval):
    mask = (pcd[:, axis] > interval[0]) & (pcd[:, axis] < interval[1])
    return pcd[mask]

pcd_masked = passthrough_filter(pcd, axis=2, interval=[0.5, 1.0])
# pcd_masked_down = np.array(pcd_masked)[::50]  # downsample to visualize with markers for speed.

plotter = pv.Plotter()
plotter.add_points(p_semicircle)
plotter.add_points(p_obj, color="red")
plotter.show()

Widget(value="<iframe src='http://localhost:33007/index.html?ui=P_0x7fd264f89dc0_10&reconnect=auto' style='wid…

In [86]:
sampled_idx = np.random.randint(0, len(p_semicircle), 1)[0]
p_cand = p_semicircle[sampled_idx]
R_cand = R_semicirlce[sampled_idx]

env.init_viewer(viewer_title='UR5e with RG2 gripper',viewer_width=1200,viewer_height=800,
                viewer_hide_menus=True)
env.update_viewer(azimuth=0.0,distance=3.5,elevation=-60,lookat=[0.4,0.05,0.36],
                  VIS_TRANSPARENT=False,VIS_CONTACTPOINT=True,
                  contactwidth=0.05,contactheight=0.05,contactrgba=np.array([1,0,0,1]),
                  VIS_JOINT=True,jointlength=0.5,jointwidth=0.1,jointrgba=[0.2,0.6,0.8,0.6])
env.reset()

init_ur_q = np.array([np.deg2rad(-90), np.deg2rad(-130), np.deg2rad(120), np.deg2rad(100), np.deg2rad(45), np.deg2rad(-90)])
q_init_upright = np.array([0,-np.pi/2,0,0,np.pi/2,0])
pre_grasp_pose = p_cand.copy()
pre_grasp_pose = pre_grasp_pose + normalized_projected_vec * 0.15
p_grasp = p_obj.copy()
p_grasp = p_grasp - normalized_projected_vec * 0.05

q_ik_pregrasp = env.solve_ik_repel(
    body_name='tcp_link',p_trgt=pre_grasp_pose,R_trgt=R_cand,
    IK_P=True,IK_R=True, q_init=q_init_upright,idxs_forward=env.idxs_forward, idxs_jacobian=env.idxs_jacobian,
    RESET=False, DO_RENDER=True, th=1, err_th=1e-3, stepsize=1.0, w_weight=0.3, render_every=1.0, eps = 0.3, repulse = 15, VERBOSE=False)

q_ik = env.solve_ik_repel(
    body_name='tcp_link',p_trgt=p_cand,R_trgt=R_cand,
    IK_P=True,IK_R=True, q_init=q_ik_pregrasp,idxs_forward=env.idxs_forward, idxs_jacobian=env.idxs_jacobian,
    RESET=False, DO_RENDER=True, th=1, err_th=1e-3, stepsize=1.0, w_weight=0.3, render_every=1.0, eps = 0.3, repulse = 15, VERBOSE=False)

q_pick = env.solve_ik_repel(
    body_name='tcp_link',p_trgt=p_grasp,R_trgt=R_cand,
    IK_P=True,IK_R=True, q_init=q_ik_pregrasp,idxs_forward=env.idxs_forward, idxs_jacobian=env.idxs_jacobian,
    RESET=False, DO_RENDER=True, th=1, err_th=1e-3, stepsize=1.0, w_weight=0.3, render_every=1.0, eps = 0.3, repulse = 15, VERBOSE=False)

print(f"Solved joint q values: {q_ik}")
# Close viewer
env.close_viewer()
print ("Done.")

q_init_upright = np.array([0,-np.pi/2,0,0,np.pi/2,0])
times, q_traj = get_interp_const_vel_traj(np.vstack([q_init_upright,q_ik_pregrasp, q_ik, q_pick]), vel=np.radians(30), HZ=env.HZ)
print ("Joint trajectory ready. duration:[%.2f]sec"%(times[-1]))

Solved joint q values: [ 0.18 -1.08  2.18 -1.1   1.94 -0.  ]
Done.
Joint trajectory ready. duration:[5.74]sec


In [90]:
# Init viewer
env.init_viewer(viewer_title='UR5e with RG2 gripper',viewer_width=1200,viewer_height=800,
                viewer_hide_menus=True)
env.update_viewer(azimuth=66.08,distance=3.0,elevation=-50,lookat=[0.4,0.18,0.71],
                  VIS_TRANSPARENT=False,VIS_CONTACTPOINT=False,
                  contactwidth=0.05,contactheight=0.05,contactrgba=np.array([1,0,0,1]),
                  VIS_JOINT=True,jointlength=0.25,jointwidth=0.05,jointrgba=[0.2,0.6,0.8,0.6])

pcd_tick = 0

# Reset
env.reset()
q_init_upright = np.array([0,-np.pi/2,0,0,np.pi/2,0])
env.forward(q=q_init_upright, joint_idxs=env.idxs_forward)
tick,max_sec = 0,1000
while env.get_sim_time() <= max_sec:
    # q = q_traj[tick, :]
    # Step
    if np.linalg.norm(env.get_p_body('tcp_link')-p_grasp) < 0.0125:
        q = np.append(q_traj[tick, :],0.0) # close gripper
    else:
        q = np.append(q_traj[tick, :],1.0) # open gripper
    env.step(ctrl=q,ctrl_idxs=[0,1,2,3,4,5,6])

    tick = min(tick + 1,q_traj.shape[0]-1)
    if not env.is_viewer_alive(): break
        
    # Render
    if env.loop_every(HZ=20):
        # Compute some poses
        p_tcp,R_tcp = env.get_pR_body(body_name='tcp_link')
        p_cam,R_cam = env.get_pR_body(body_name='camera_center')
        p_base,R_base = env.get_pR_body(body_name='base')
        # Get PCD from a specific view
        p_ego  = p_cam
        p_trgt = p_cam + R_cam[:,2]
        rgb_img,depth_img,pcd,xyz_img = env.get_egocentric_rgb_depth_pcd(
            p_ego=p_ego,p_trgt=p_trgt,rsz_rate=40,fovy=45,BACKUP_AND_RESTORE_VIEW=True)
        env.plot_T(p=p_base+1.25*R_base[:,2],R=np.eye(3,3),
                    PLOT_AXIS=False,label='[%.4f] err'%(np.linalg.norm(env.get_p_body('tcp_link')-p_grasp)))
        for i in range(len(p_semicircle)):
            env.plot_sphere(p=p_semicircle[i], r=0.005, rgba=[0,1,0,1])
        env.render(render_every=1)
        
# Close viewer
env.close_viewer()
print ("Done. Tick:[%d] Time:[%.2f]sec"%(env.tick,env.get_sim_time()))

Pressed ESC
Quitting.
Done. Tick:[5702] Time:[11.40]sec


## Detect Grasp pose

In [13]:
import cv2
import numpy as np

def get_grasp_pose(image_array):
    # Copy the array to avoid modifying the original image
    image = np.copy(image_array)
    # Convert image to grayscale
    grayscale_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Apply thresholding to get binary image
    _, binary_image = cv2.threshold(grayscale_image, 128, 255, cv2.THRESH_BINARY)
    # Find contours
    contours, _ = cv2.findContours(binary_image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Iterate through contours and find top points
    for contour in contours:
        # Get the bounding rect
        x, y, w, h = cv2.boundingRect(contour)
        # Top point is the first point in the bounding rect
        top_point = (x + w//2, y)
        # Draw circle for grasp point
        cv2.circle(image, top_point, 5, (0, 255, 0), -1)
        # Draw rectangle for object
        cv2.rectangle(image, (x, y), (x+w, y+h), (255, 0, 0), 2)

    # Show the image
    plt.imshow(image)
    plt.show()
    
# Test the function
# get_grasp_pose(rgb_img)
